In [531]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.quantum_info import Statevector, Operator, partial_trace
from qiskit.circuit.library.standard_gates import RYGate
import matplotlib.pyplot as plt
import numpy as np
import pylatexenc

In [532]:
def make_circuit(psi):
    def make_list(L,n1):
        stupid = []
        for j1 in range(L):
            nr = ''.join(reversed(format(j1,'0'+str(n1)+'b')))
            stupid.append(nr)
        return stupid

    def get_angles(state_0):
        st = make_list(len(state_0),n)
        angles = []
        newstate = []
        for j1 in range(len(st)//2):
            angles.append(2*np.arccos(state_0[2*j1]/np.sqrt(state_0[2*j1]**2+state_0[2*j1+1]**2)))
            newstate.append(np.sqrt(state_0[2*j1]**2+state_0[2*j1+1]**2))
        return angles, newstate


    L = len(psi)
    n = int(np.log2(L))

    tot_angles=[] #get all the angles we'll need
    [angles,newstate] = get_angles(psi)
    tot_angles.append(angles)
    for j1 in range(n-1):
        [angles,newstate] = get_angles(newstate)
        tot_angles.append(angles)
    tot_angles = list(reversed(tot_angles))

    dumb = []
    for j1 in range(1,n+1):
        nq = n-j1
        dumb1 = []
        for j in range(2**(n-j1)):
            n1=format(j,'0'+str(nq)+'b')
            dumb1.append(n1)
        dumb.append(dumb1)
    dumb=list(reversed(dumb))


    q_reg = QuantumRegister(n, name = "x") #we need as many quantum bits as we need
    U_circ = QuantumCircuit(q_reg, name = "Quantum State Preparation") #initialize the circuit
    U_circ.ry(tot_angles[0][0],q_reg[n-1])

    for j1 in range(1,len(tot_angles)):
        curr_angles = tot_angles[j1]
        for j2 in range(len(curr_angles)):
            now_angle = curr_angles[j2]
            for j3 in range(len(dumb[j1][j2])):
                if dumb[j1][j2][j3] == '0':
                    U_circ.x(q_reg[n-1-j3])

            U_circ.append(RYGate(now_angle).control(j1),q_reg[::-1][0:j1+1])

            for j3 in range(len(dumb[j1][j2])):
                if dumb[j1][j2][j3] == '0':
                    U_circ.x(q_reg[n-1-j3])

    return U_circ

In [533]:
n = 3
psi = (1 + np.arange(2**n)) / np.linalg.norm(1 + np.arange(2**n))
print('|ψ>:',psi,'\n|| |ψ> ||_2:', np.linalg.norm(psi))

qc=make_circuit(psi)

st=Statevector(qc)
np.linalg.norm(st.data - psi)

|ψ>: [0.070014   0.14002801 0.21004201 0.28005602 0.35007002 0.42008403
 0.49009803 0.56011203] 
|| |ψ> ||_2: 0.9999999999999999


np.float64(2.0955000055363631e-16)